In [1]:
#import dataset class
from dataset import energyProject_dataset
from torch.utils.data import DataLoader
import pickle
import torch
import torch.nn as nn
import numpy as np
import tqdm
import torch
import torch.nn as nn
import pandas as pd
import yaml

In [2]:
#load dataset object file
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

#now you can create a dataloader and use it!
batch_size = 100
dataloader = DataLoader(dataset, batch_size, shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset_class.pkl'

In [8]:
model = torch.load("opti_model_0.pt")
model

RuntimeError: torch.UntypedStorage(): Storage device not recognized: mps

In [ ]:
hourly_activities = [ocu_emb for ocu_emb, _, _  in dataset]
hourly_conditions = [general_data for _, _, general_data in dataset]
all_classrooms = list(set([classroom for _, hourly_classroom, _ in dataset for classroom in hourly_classroom]))

# 1r: mirar quina classe resulta mes buida

In [ ]:
device = "cpu"

with torch.no_grad():
    for idx, batch in dataloader:   
                